<font color='red'>
    .به دلیل این که نقشه ها به صورت تعاملی می باشند در صورتی که نقشه ای را نمی بینید باید نوت بوک را اجرا کنید
    .با کلیک بر روی هر نقطه میتوانید شماره سفارش و شماره خوشه آن را مشاهده کنید
</font>

# Imports

## pakages

In [ ]:
# Data Handling
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import folium as fo

# Utils
import os
import copy
import random

# model
from sklearn.cluster import KMeans

In [ ]:
class cfg:
    base_dir = os.getcwd()
    data_dir = f'{os.getcwd()}\\Data\\DC-DATA.xlsx'
    color = sns.color_palette()

## Data

In [ ]:
df = pd.read_excel(cfg.data_dir, sheet_name=1)
df.tail()

In [ ]:
df.dtypes

In [ ]:
df = df.set_index('شماره سفارش')
df.head()

In [ ]:
lats=[]
longs=[]
for l in df['Lat Long'].values:
    lat, long = l[6:-1].split(' ')
    lats.append(float(lat))
    longs.append(float(long))

df['Lats'] = lats
df['Longs'] = longs
df = df.drop(['Lat Long'], axis=1)

In [ ]:
df.head()

In [ ]:
tehran = fo.Map(location=[35.7219, 51.3347], zoom_start=11)
for (i,lat,long) in zip(df.index.values, df.Lats, df.Longs):
    fo.CircleMarker(location=(long,lat), radius=3, popup=i, fill=True).add_to(tehran)

In [ ]:
tehran

In [ ]:
n = 50  # number of clusters: number of operational vehicles

model = KMeans(n_clusters=n)
model.fit(df[['Lats', 'Longs']])

In [ ]:
color_pal = []
for i in range(n):
    hex_color = "#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])
    color_pal.append(hex_color)

color_pal = sns.color_palette(color_pal)
color_pal = color_pal.as_hex()

df['cluster'] = model.labels_

centroids = model.cluster_centers_
max_d = []
for i, c in enumerate(centroids):
    d = []
    for p in df.loc[df.cluster==i].values:
        l = (p[0], p[1])
        d.append(np.linalg.norm(l-c))
    max_d.append(max(d))

In [ ]:
tehran = fo.Map(location=[35.7219, 51.3347], zoom_start=11)

for i, r in enumerate(max_d):
    fo.Circle(location=np.flip(centroids[i]), 
                    radius=(r+0.015)*1e5, 
                    color='grey',
                    fill=False).add_to(tehran)

for (i,lat,long,cluster) in zip(df.index.values, df.Lats, df.Longs, df.cluster):
    fo.CircleMarker(location=(long,lat), 
                    radius=3, 
                    popup=i, 
                    color=color_pal[cluster], 
                    tooltip='cluster No. '+str(cluster), 
                    fill=True).add_to(tehran)


In [ ]:
tehran

In [ ]:
color_pal

In [ ]:
df